imports/loads

In [5]:
import open3d as o3d
import numpy as np

# Load the stump and socket meshes
stump_mesh = o3d.io.read_triangle_mesh("stump.stl")
socket_mesh = o3d.io.read_triangle_mesh("socket.stl")

# Check if meshes are valid
print("Before cleaning:")
print("Stump Mesh:", stump_mesh)
print("Socket Mesh:", socket_mesh)

# Clean the meshes
stump_mesh.remove_duplicated_vertices()
stump_mesh.remove_degenerate_triangles()
stump_mesh.remove_duplicated_triangles()
stump_mesh.remove_non_manifold_edges()

socket_mesh.remove_duplicated_vertices()
socket_mesh.remove_degenerate_triangles()
socket_mesh.remove_duplicated_triangles()
socket_mesh.remove_non_manifold_edges()

print("After cleaning:")
print("Stump Mesh:", stump_mesh)
print("Socket Mesh:", socket_mesh)

# Sample points uniformly from the cleaned meshes
stump_pcd = stump_mesh.sample_points_uniformly(number_of_points=3000)
socket_pcd = socket_mesh.sample_points_uniformly(number_of_points=3000)

# Visualize the original mesh and sampled points
o3d.visualization.draw_geometries([stump_mesh, stump_pcd], window_name="Stump Mesh and Points")
o3d.visualization.draw_geometries([socket_mesh, socket_pcd], window_name="Socket Mesh and Points")

Before cleaning:
Stump Mesh: TriangleMesh with 8421 points and 16821 triangles.
Socket Mesh: TriangleMesh with 804 points and 666 triangles.
After cleaning:
Stump Mesh: TriangleMesh with 8421 points and 16817 triangles.
Socket Mesh: TriangleMesh with 335 points and 666 triangles.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2024-12-08 00:30:46.355 python[27338:1464435] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-08 00:30:46.355 python[27338:1464435] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Resize so that the scales are similar
Modify the "margin" parameter

In [6]:
# Function to get the size of a mesh
def get_mesh_size(mesh):
    min_bound = mesh.get_min_bound()
    max_bound = mesh.get_max_bound()
    size = max_bound - min_bound  # Size along X, Y, Z
    return size

# Function to find the maximum radius of the stump
def find_max_radius(mesh):
    vertices = np.asarray(mesh.vertices)
    # Compute the distance of each vertex from the Y-axis (radius in X-Z plane)
    radii = np.sqrt(vertices[:, 0]**2 + vertices[:, 2]**2)  # sqrt(x^2 + z^2)
    max_radius = np.max(radii)  # Maximum radius
    print(f"Maximum Radius: {max_radius}")
    return max_radius

# Algorithm to adjust stump and socket radii with a fixed margin
def adjust_radii_with_fixed_margin(stump_mesh, socket_mesh, margin):
    # Find the maximum radius of the stump
    stump_max_radius = find_max_radius(stump_mesh)

    # Get the average radius of the socket
    socket_size = get_mesh_size(socket_mesh)
    socket_radius = (socket_size[0] + socket_size[2]) / 2

    print(f"Initial Stump Max Radius: {stump_max_radius}")
    print(f"Initial Socket Radius: {socket_radius}")

    # Calculate the desired socket radius
    desired_socket_radius = stump_max_radius + margin

    # Adjust the socket to match the desired radius
    if abs(socket_radius - desired_socket_radius) > 1e-6:  # Avoid tiny floating-point differences
        scale_factor = desired_socket_radius / socket_radius
        socket_mesh.scale(scale_factor, center=(0, 0, 0))  # Scale around the origin
        print(f"Scaled the socket by factor: {scale_factor}")

    # Recalculate the socket radius after scaling
    socket_size = get_mesh_size(socket_mesh)
    socket_radius = (socket_size[0] + socket_size[2]) / 2

    print(f"Final Stump Max Radius: {stump_max_radius}")
    print(f"Final Socket Radius: {socket_radius}")
    print(f"Margin: {socket_radius - stump_max_radius} (should be {margin})")

    return stump_mesh, socket_mesh

# Set the desired margin
margin = 0.1  # The fixed difference between the socket radius and the stump max radius

# Adjust the stump and socket radii to respect the margin
adj_stump_mesh, adj_socket_mesh = adjust_radii_with_fixed_margin(stump_mesh, socket_mesh, margin)

Maximum Radius: 0.5883575332901089
Initial Stump Max Radius: 0.5883575332901089
Initial Socket Radius: 147.48076629638672
Scaled the socket by factor: 0.004667439358883869
Final Stump Max Radius: 0.5883575332901089
Final Socket Radius: 0.6883575332901088
Margin: 0.09999999999999998 (should be 0.1)


aligns + merges them 
Play with the "offset" parameter to ajust the depth of the stu,p in the socket

In [7]:
# Function to align a mesh to the Y-axis
def align_to_y_axis(mesh):
    vertices = np.asarray(mesh.vertices)
    center_of_mass = np.mean(vertices, axis=0)  # Center of mass
    centered_vertices = vertices - center_of_mass  # Centered vertices
    cov = np.cov(centered_vertices.T)  # Covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(cov)  # Eigen decomposition

    # Extract the first principal component
    principal_axis = eigenvectors[:, np.argmax(eigenvalues)]  # Principal axis
    print("Principal Component Axis:", principal_axis)

    # Compute rotation matrix to align the principal axis with the Y-axis
    y_axis = np.array([0, 1, 0])  # Target Y-axis
    v = np.cross(principal_axis, y_axis)  # Cross product for the rotation axis
    s = np.linalg.norm(v)  # Magnitude of the rotation axis
    if s < 1e-6:
        print("Mesh is already aligned to the Y-axis.")
        return mesh  # No rotation needed

    c = np.dot(principal_axis, y_axis)  # Dot product for the cosine of the angle
    v_skew = np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])  # Skew-symmetric matrix for cross product
    rotation_matrix = np.eye(3) + v_skew + np.dot(v_skew, v_skew) * ((1 - c) / (s ** 2))

    # Apply the rotation
    mesh.rotate(rotation_matrix, center=center_of_mass)

    # Translate the mesh to align its center of mass in the X-Z plane
    mesh.translate((-center_of_mass[0], 0, -center_of_mass[2]))
    return mesh

# Align both meshes to the Y-axis
stump_mesh_aligned = align_to_y_axis(adj_stump_mesh)
socket_mesh_aligned = align_to_y_axis(adj_socket_mesh)

# Get the height of the socket
socket_height = socket_mesh_aligned.get_max_bound()[1] - socket_mesh_aligned.get_min_bound()[1]

# Position the stump below the socket
offset = -0.1  # Adjust this offset as needed
stump_mesh_aligned.translate((0, -socket_height - offset, 0))


stump_aligned_pcd = stump_mesh_aligned.sample_points_uniformly(number_of_points=3000)
socket_aligned_pcd = socket_mesh_aligned.sample_points_uniformly(number_of_points=3000)
# Visualize both meshes
print("Displaying adjusted and aligned socket and stump")
o3d.visualization.draw_geometries([stump_mesh_aligned, socket_mesh_aligned], window_name="Stump Below Socket")
o3d.visualization.draw_geometries([stump_aligned_pcd, socket_aligned_pcd], window_name="Stump Below Socket")

Principal Component Axis: [0.13612773 0.88551163 0.44422785]
Principal Component Axis: [-4.54222915e-19  0.00000000e+00 -1.00000000e+00]
Displaying adjusted and aligned socket and stump
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


performs boolean substraction

In [8]:
import trimesh
import open3d as o3d
import numpy as np

# Function to compute the Boolean difference between two meshes
def compute_difference(stump_mesh, socket_mesh):
    # Convert Open3D meshes to Trimesh
    stump_trimesh = trimesh.Trimesh(vertices=np.asarray(stump_mesh.vertices),
                                    faces=np.asarray(stump_mesh.triangles))
    socket_trimesh = trimesh.Trimesh(vertices=np.asarray(socket_mesh.vertices),
                                    faces=np.asarray(socket_mesh.triangles))
    
    # Check if meshes are volumes
    print("Is Stump a Volume?", stump_trimesh.is_volume)
    print("Is Socket a Volume?", socket_trimesh.is_volume)

    # Fix issues if meshes are not volumes
    if not stump_trimesh.is_watertight:
        print("Fixing stump to be watertight...")
        stump_trimesh.fill_holes()
    
    if not socket_trimesh.is_watertight:
        print("Fixing socket to be watertight...")
        socket_trimesh.fill_holes()
    
    # Ensure normals are consistent
    stump_trimesh.fix_normals()
    socket_trimesh.fix_normals()
    
    # As a last resort, use convex hull to ensure both are volumes
    if not stump_trimesh.is_volume:
        print("Converting stump to convex hull...")
        stump_trimesh = stump_trimesh.convex_hull
    
    if not socket_trimesh.is_volume:
        print("Converting socket to convex hull...")
        socket_trimesh = socket_trimesh.convex_hull

    # Perform Boolean difference: Socket - Stump
    difference_trimesh = socket_trimesh.difference(stump_trimesh)
    
    # Convert the result back to Open3D for visualization
    difference_mesh = o3d.geometry.TriangleMesh()
    difference_mesh.vertices = o3d.utility.Vector3dVector(difference_trimesh.vertices)
    difference_mesh.triangles = o3d.utility.Vector3iVector(difference_trimesh.faces)
    difference_mesh.compute_vertex_normals()
    
    return difference_mesh

# Compute the difference between the socket and the stump
difference_mesh = compute_difference(stump_mesh_aligned, socket_mesh_aligned)

difference_pcd = difference_mesh.sample_points_uniformly(number_of_points=3000)
# Visualize the result
print("Displaying the difference mesh (socket minus stump)")
o3d.visualization.draw_geometries([difference_mesh], window_name="Difference Mesh")
o3d.visualization.draw_geometries([difference_pcd], window_name="Difference Mesh")
import open3d as o3d
import os
from IPython.display import FileLink

# Step 2: Save the mesh as an STL file
file_name = "difference_mesh.stl"  # Desired file name
o3d.io.write_triangle_mesh(file_name, difference_mesh)

# Step 3: Provide a download link in the Jupyter Notebook
display(FileLink(file_name))

# Optional: Save directly to desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", file_name)
os.rename(file_name, desktop_path)  # Move the file to the desktop
print(f"The file has been saved to your desktop: {desktop_path}")

Is Stump a Volume? False
Is Socket a Volume? True
Fixing stump to be watertight...
Displaying the difference mesh (socket minus stump)
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


/Users/dariusgiannoli/Documents/Npulse/difference_mesh.stl

The file has been saved to your desktop: /Users/dariusgiannoli/Desktop/difference_mesh.stl
